# Team Big HIT Stage 3 Notebook
## Introduction
This Jupyter notebook describes the steps taken in our project stage 3, which did entity matching between two tables of data generated from www.imdb.com and www.themoviedb.org. Our goal was to match as many movies as possible with at least 90% precision and  as high recall as we could achieve.

To begin, we had to install and import the py_entitymatching module and other relevant packages.

In [6]:
# import required library
import py_entitymatching as em
import pandas as pd
import numpy as np
import os, sys
import py_stringmatching as sm

## 1) Reading in input tables A, B 
In the first step towards entity matching, we had to read in the input tables from imdb and tmdb into Python. These tables were generated in project stage 2.

In [7]:
# Specify data path
folder = "../Data/"
A = pd.read_csv(folder + 'A.csv') # assuming A.csv exists
B = pd.read_csv(folder + 'B.csv') # assuming B.csv exists
# Set keys to tables
em.set_key(A, 'id')
em.set_key(B, 'id')
# View shape of A and B
print("shape of A: ", A.shape)
print("shape of B: ", B.shape)


shape of A:  (3500, 18)
shape of B:  (5490, 18)


## 2) Applying blockers on input tables to generate candidate set C

Since our tables are small, there is no need to do down-sampling. 
In the first iteration, we blocked movies that are not released in the same year using attribute-equivalence blocker


In [8]:
ab = em.AttrEquivalenceBlocker()
# block on release_year
C1 = ab.block_tables(A, B, l_block_attr='release_year', r_block_attr='release_year', 
                    l_output_attrs=['title', 'directors', 'release_year'],
                    r_output_attrs=['title', 'directors', 'release_year'],
                    l_output_prefix='l_', r_output_prefix='r_')
C1.head(5)

,_id,l_id,r_id,l_title,l_directors,l_release_year,r_title,r_directors,r_release_year
0,0,a0,b34,The Shape of Water,Guillermo del Toro,2017,Logan Lucky,Steven Soderbergh,2017
1,1,a0,b62,The Shape of Water,Guillermo del Toro,2017,The Greatest Showman,Michael Gracey,2017
2,2,a0,b65,The Shape of Water,Guillermo del Toro,2017,American Made,Doug Liman,2017
3,3,a0,b82,The Shape of Water,Guillermo del Toro,2017,Alibi.com,Philippe Lacheau,2017
4,4,a0,b133,The Shape of Water,Guillermo del Toro,2017,Dunkirk,Christopher Nolan,2017


In [9]:
# Shape of C1
C1.shape

(380818, 9)

The candidate set has too many tuples, we needed to block more tables. We then blocked movies that do not have the same title by applying attribute-equivalence blocker on C1. 

In [10]:
# block on candidate set C1 on title
C2 = ab.block_candset(C1, 'title', 'title', show_progress=False)
C2.head(5)

,_id,l_id,r_id,l_title,l_directors,l_release_year,r_title,r_directors,r_release_year
127,127,a0,b3131,The Shape of Water,Guillermo del Toro,2017,The Shape of Water,Guillermo del Toro,2017
265,265,a2,b1130,"Three Billboards Outside Ebbing, Missouri",Martin McDonagh,2017,"Three Billboards Outside Ebbing, Missouri",Martin McDonagh,2017
447,447,a3,b262,Call Me by Your Name,Luca Guadagnino,2017,Call Me by Your Name,Luca Guadagnino,2017
815,815,a4,b4046,Get Out,Jordan Peele,2017,Get Out,Jordan Peele,2017
914,914,a5,b1054,Thor: Ragnarok,Taika Waititi,2017,Thor: Ragnarok,Taika Waititi,2017


In [11]:
C2.shape

(1038, 9)

This blocker maybe too aggressive so we looked at some of the outputs of the blocker debugger. 

In [12]:
dbq = em.debug_blocker(C2, A, B, output_size = 100)
dbq.head(5)

,_id,ltable_id,rtable_id,ltable_title,ltable_cast,ltable_directors,ltable_writers,ltable_genres,ltable_keywords,ltable_content_rating,...,rtable_genres,rtable_keywords,rtable_content_rating,rtable_languages,rtable_budget,rtable_revenue,rtable_opening_weekend_revenue,rtable_production_companies,rtable_production_countries,rtable_alternative_titles
0,0,a198,b550,Pirates of the Caribbean: The Curse of the Black Pearl,Johnny Depp;Geoffrey Rush;Orlando Bloom;Keira Knightley;Jack Davenport,Gore Verbinski,Ted Elliott;Terry Rossio,Action;Adventure;Fantasy,curse;undead;pirate;governor;caribbean,PG-13,...,Adventure;Fantasy;Action,witch;fortune teller;bondage;exotic island;monster;captain;card game;east india trading company;...,PG-13,English,$200000000.00,$1065659812.00,NaN,NaN,NaN,NaN
1,1,a1093,b1635,The Meaning of Life,Graham Chapman;John Cleese;Terry Gilliam;Eric Idle;Terry Jones,Terry Jones;Terry Gilliam,Graham Chapman;John Cleese,Comedy;Musical,death character;monty python;meaning of life;sperm;grim reaper,R,...,Comedy,christianity;jewry;resistance;gladiator;roman empire;balcony;miracle;graffiti;jesus christ;doome...,R,English,$4000000.00,$20045115.00,NaN,NaN,NaN,NaN
2,2,a67,b5358,Goodfellas,Robert De Niro;Ray Liotta;Joe Pesci;Lorraine Bracco;Paul Sorvino,Martin Scorsese,Nicholas Pileggi;Nicholas Pileggi,Biography;Crime;Drama,gangster;rise and fall;organized crime;mafia;cold blooded murder,R,...,Drama;Crime,prison;based on novel;florida;1970s;mass murder;irish-american;drug traffic;gangster;biography;b...,R,English,$25000000.00,$46836394.00,NaN,NaN,NaN,NaN
3,3,a778,b578,The Naked Gun: From the Files of Police Squad!,Leslie Nielsen;Priscilla Presley;Ricardo Montalban;George Kennedy;O.J. Simpson,David Zucker,Jerry Zucker;Jim Abrahams,Comedy;Crime,assassin;terrorist;good versus evil;police;queen,PG-13,...,Comedy;Crime,washington d.c.;usa president;spoof;the white house,PG-13,English,$23000000.00,$86930411.00,NaN,NaN,NaN,NaN
4,4,a898,b2223,Rashômon,Toshirô Mifune;Machiko Kyô;Masayuki Mori;Takashi Shimura;Minoru Chiaki,Akira Kurosawa,Ryûnosuke Akutagawa;Akira Kurosawa,Crime;Drama;Mystery,unreliable narration;heavy rain;contradictory accounts;multiple perspectives;story told from dif...,NOT RATED,...,Crime;Drama;Mystery,japan;rape;samurai;rain;woodcutter;medium;sunlight;court case;dying and death;court;truth;crimin...,NR,Japanese,$250000.00,$96568.00,NaN,NaN,NaN,NaN


The movies a67 and b5358 are actually the same movie but in IMDb, its title is 'Goodfellas', while in TMDb, it is 'Good Fellas'. It has the same cast and crew and was released in the same year and yet it was blocked. This blocker is too restricted so we instead blocked movies that do not have at least one match in one of the attributes 'directors', 'writers', and 'cast'. However, since there could be more than one director, writer, and actor in each of these fields, we used a black-box blocker in which we implemented our own blocking criteria defined in function below. Specifically, for a pair of movies, we extracted directors, writers and cast to 3 separate lists. We iterate through the directors' list for each tuple and compute the similarity score between each pair of directors. If the score is greater than 0.8 (almost exact match), then we set a flag indicating there is a match in director name to True. We did the same for the other lists. If one of these flags is True, the tuple pair survives the blocking rules, otherwise, it would be blocked. 

In [13]:
def blocking_rules(x, y):
    # return True if x and y survive the blocking rules
    # x and y are pandas series
    x_directors = str(x['directors']).split(';')
    y_directors = str(y['directors']).split(';')
    
    x_writers = str(x['writers']).split(';')
    y_writers = str(y['writers']).split(';')
    x_actors = str(x['cast']).split(';')
    y_actors = str(y['cast']).split(';')
    director_match = False
    writer_match = False
    actor_match = False
    overlap_size = 0
    # create a tokenizer
    ws_tok = sm.WhitespaceTokenizer()
    # create a Jaccard similarity measure object
    jac = sm.Jaccard()
    for x_director in x_directors:
        if director_match == True:
            break
        else:
            # tokenize x_director using whitespace
            if x_director == 'nan':
                continue
            else:
                x_director = ws_tok.tokenize(x_director)
                for y_director in y_directors:
                    if y_director == 'nan':
                        continue
                    else:
                        # tokenize y_director using whitespace
                        y_director = ws_tok.tokenize(y_director)
                        if jac.get_sim_score(x_director, y_director) >= 0.8:
                            director_match == True
                            break
    for x_writer in x_writers:
        if writer_match == True:
            break
        else:
            if x_writer == 'nan':
                continue    
            else:
                x_writer = ws_tok.tokenize(x_writer)
                for y_writer in y_writers:
                    if y_writer == 'nan':
                        continue
                    else:
                        y_writer = ws_tok.tokenize(y_writer)
                        if jac.get_sim_score(x_writer, y_writer) >= 0.8:
                            writer_match = True
                            break
    for x_actor in x_actors:
        if actor_match == True:
            break
        else:
            if x_actor == 'nan':
                continue
            else:
                x_actor = ws_tok.tokenize(x_actor)
                for y_actor in y_actors:
                    if y_actor == 'nan':
                        continue
                    else:
                        y_actor = ws_tok.tokenize(y_actor)
                        if jac.get_sim_score(x_actor, y_actor) >= 0.8:
                            actor_match = True
                            break
    if actor_match == False and director_match == False and writer_match == False:
        return True
    else:
        return False 

In [14]:
# call the black-box blocker using blocking rules above, this will take a while
bb = em.BlackBoxBlocker()
bb.set_black_box_function(blocking_rules)
C3 = bb.block_candset(C1, show_progress=False )
C3.head(5)

,_id,l_id,r_id,l_title,l_directors,l_release_year,r_title,r_directors,r_release_year
13,13,a0,b262,The Shape of Water,Guillermo del Toro,2017,Call Me by Your Name,Luca Guadagnino,2017
39,39,a0,b944,The Shape of Water,Guillermo del Toro,2017,The Shack,Stuart Hazeldine,2017
127,127,a0,b3131,The Shape of Water,Guillermo del Toro,2017,The Shape of Water,Guillermo del Toro,2017
147,147,a0,b3644,The Shape of Water,Guillermo del Toro,2017,Paddington 2,Paul King,2017
149,149,a0,b3685,The Shape of Water,Guillermo del Toro,2017,Gifted,Marc Webb,2017


In [15]:
C3.shape

(2547, 9)

Since there are 1038 movies with exact title match, we expecting the number of true matched movie pairs are approximately 1000 - 1200. Because movies with the same title may not be the same movies, and vice versa movies with different titles can be the same movies, the number of true matches will not be too off from 1038. Therefore, a candidate set of size 2547 seems reasonable to us since it is roughly twice as many as the number of the expected true matches. Let's check if we retrieve as many movies with exact title match as 1038. 

In [16]:
C4 = ab.block_candset(C3, 'title', 'title', show_progress = False)
C4.shape

(1036, 9)

Indeed we managed to recover 1036 tuple pairs with exact title match. Let's look at some of the debugger outputs

In [17]:
dbq = em.debug_blocker(C3, A, B, output_size = 100)
dbq.head(5)

,_id,ltable_id,rtable_id,ltable_title,ltable_cast,ltable_directors,ltable_writers,ltable_genres,ltable_keywords,ltable_content_rating,...,rtable_genres,rtable_keywords,rtable_content_rating,rtable_languages,rtable_budget,rtable_revenue,rtable_opening_weekend_revenue,rtable_production_companies,rtable_production_countries,rtable_alternative_titles
0,0,a372,b4949,Star Wars: Episode III - Revenge of the Sith,Ewan McGregor;Natalie Portman;Hayden Christensen;Ian McDiarmid;Samuel L. Jackson,George Lucas,George Lucas,Action;Adventure;Fantasy;Sci-Fi,elongated cry of no;vision;robotic midwife;friends become enemies;kicked in the butt,PG-13,...,Adventure;Action;Science Fiction,prophecy;senate;queen;taskmaster;galaxy;apprentice;taxes;space opera,PG,English,$115000000.00,$924317558.00,NaN,NaN,NaN,NaN
1,1,a117,b1020,Back to the Future,Michael J. Fox;Christopher Lloyd;Lea Thompson;Crispin Glover;Thomas F. Wilson,Robert Zemeckis,Robert Zemeckis;Bob Gale,Adventure;Comedy;Sci-Fi,time travel;delorean;time travel comedy;time machine;future,PG,...,Adventure;Comedy;Family;Science Fiction,railroad robber;california;delorean;indian territory;sports car;inventor;locomotive;saloon;horse...,PG,English,$40000000.00,$244527583.00,NaN,NaN,NaN,NaN
2,2,a156,b425,Star Trek,Chris Pine;Zachary Quinto;Leonard Nimoy;Eric Bana;Bruce Greenwood,J.J. Abrams,Roberto Orci;Alex Kurtzman,Action;Adventure;Sci-Fi,vulcan;future;based on tv series;lifted by the throat;star trek,PG-13,...,Action;Adventure;Science Fiction,spacecraft;friendship;sequel;futuristic;space;alien;imax;space opera;terrorist bombing;3d,PG-13,English,$190000000.00,$467365246.00,NaN,NaN,NaN,NaN
3,3,a117,b5341,Back to the Future,Michael J. Fox;Christopher Lloyd;Lea Thompson;Crispin Glover;Thomas F. Wilson,Robert Zemeckis,Robert Zemeckis;Bob Gale,Adventure;Comedy;Sci-Fi,time travel;delorean;time travel comedy;time machine;future,PG,...,Adventure;Comedy;Family;Science Fiction,skateboarding;flying car;car race;delorean;lightning;almanac;inventor;time travel;sequel;diner;a...,PG,English,$40000000.00,$332000000.00,NaN,NaN,NaN,NaN
4,4,a1093,b5037,The Meaning of Life,Graham Chapman;John Cleese;Terry Gilliam;Eric Idle;Terry Jones,Terry Jones;Terry Gilliam,Graham Chapman;John Cleese,Comedy;Musical,death character;monty python;meaning of life;sperm;grim reaper,R,...,Adventure;Comedy;Fantasy,holy grail;monk;england;scotland yard;swordplay;camelot;round table;chapter;animal attack;knight...,PG,English,$400000.00,$5028948.00,NaN,NaN,NaN,NaN


At this point, we decided our blocking step is completed because we have reduced the number of candidates to a reasonable size of 2547 with number of potential true matches ~ 1000, and the pairs that got blocked seem legit. 

## 3) Reading in labeled sample G

After generating candidate set C, we sampled a smaller set of tuples, S of 400 tuple pairs, and then labeled them as matches (1) or not-matches (0) to generate a labeled sample G.

Some examples of tuples in the labeled sample G include:

In [18]:
# labeled data
G = em.read_csv_metadata(folder+'G.csv', key = '_id', ltable = A, rtable = B,
                             fk_ltable = 'l_id', fk_rtable = 'r_id') 
G.head(3)

Metadata file is not present in the given path; proceeding to read the csv file.


,_id,l_id,r_id,l_title,l_cast,l_directors,l_writers,l_genres,l_keywords,l_content_rating,...,r_release_year,r_languages,r_rating,r_budget,r_revenue,r_opening_weekend_revenue,r_production_companies,r_production_countries,r_alternative_titles,label
0,149,a0,b3685,The Shape of Water,Sally Hawkins;Michael Shannon;Richard Jenkins;Octavia Spencer;Michael Stuhlbarg,Guillermo del Toro,Guillermo del Toro;Vanessa Taylor,Adventure;Drama;Fantasy;Horror;Romance;Thriller,underwater scene;mute woman;creature;interspecies romance;sign language,R,...,2017,English,78,"$7,000,000.00","$40,343,446.00",NaN,NaN,NaN,NaN,0
1,593,a3,b3968,Call Me by Your Name,Armie Hammer;Timothée Chalamet;Michael Stuhlbarg;Amira Casar;Esther Garrel,Luca Guadagnino,James Ivory;André Aciman,Drama;Romance,father son relationship;male male kiss;gay romance;american abroad;sex with fruit,R,...,2017,English,68,"$175,000,000.00","$383,715,756.00",NaN,NaN,NaN,NaN,0
2,887,a5,b412,Thor: Ragnarok,Chris Hemsworth;Tom Hiddleston;Cate Blanchett;Idris Elba;Jeff Goldblum,Taika Waititi,Eric Pearson;Craig Kyle,Action;Adventure;Comedy;Fantasy;Sci-Fi,marvel cinematic universe;norse god;marvel comics;female villain;superhero,PG-13,...,2017,English,63,"$185,000,000.00","$566,652,812.00",NaN,NaN,NaN,NaN,0


## 4) Splitting the labeled set G into development set I and evaluation set J
We divided the labeled data equally to set I which is used to select the best matcher and set J, which is used to evaluate the selected matcher's prediction. 

In [19]:
IJ = em.split_train_test(G, train_proportion = 0.5, random_state = 0)
I = IJ['train']
J = IJ['test']

## 5) Create a set of machine learning matchers

In [20]:
# Create a set of machine-learning matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

## 6) Select the best matcher using I
We performed cross validation using the matchers above on set I, and selected the best matcher to be the one with highest precision score. 

In [21]:
# Generate features set F
F = em.get_features_for_matching(A, B, validate_inferred_attr_types = False)
# Convert I to a set of feature vectors using F
H = em.extract_feature_vecs(I, feature_table = F, attrs_after = 'label', show_progress = False)
# Fill in missing values with column's average
excluded_attributes = ['_id', 'l_id', 'r_id', 'label']
H = em.impute_table(H, exclude_attrs = excluded_attributes, strategy='mean')
# Selecting best matcher with CV using F1-score as criteria
CV_result = em.select_matcher([dt, rf, svm, ln, lg, nb], table = H,
                                  exclude_attrs = excluded_attributes,
                                  k = 10, target_attr = 'label',
                                  metric_to_select_matcher = 'f1',
                                  random_state = 0)
# View results
CV_result['cv_stats']

Column opening_weekend_revenue does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column opening_weekend_revenue
Column production_companies does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column production_companies
Column production_countries does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column production_countries
Column alternative_titles does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column alternative_titles


,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.965000,0.980000,0.969925
1,RF,0.974603,0.990000,0.981162
2,SVM,0.900000,0.175952,0.287214
3,LinReg,0.963889,0.960000,0.958779
4,LogReg,0.978889,0.980000,0.978328
5,NaiveBayes,0.964603,1.000000,0.981162


Based on the results above, all classifier except SVM meet the classification requirement of at least 90% precision and high recall and therefore we don't need to do any debugging. Logistic regression classifier gave the highest precision score so we selected it as our final best matcher (matcher Y)

## 7) Evaluate the best matcher Y using evaluation set J
We trained logistic regresion classifier on set I and applied the trained classifier on set J to make predictions on whether the tuple pair matches. 

In [22]:
# Train the classifier on I
lg.fit(table = H, exclude_attrs = excluded_attributes, target_attr = 'label')
# Convert J into a set of features using F
L = em.extract_feature_vecs(J, feature_table = F, attrs_after = 'label', show_progress = False)
# Fill in missing values with column's average
L = em.impute_table(L, exclude_attrs = excluded_attributes, strategy='mean')
# Predict on L with lg
predictions_lg = lg.predict(table = L, exclude_attrs = excluded_attributes, 
                             append = True, target_attr = 'predicted', inplace = False,
                             return_probs = True, probs_attr = 'proba')
# Evaluate predictions
lg_eval = em.eval_matches(predictions_lg, 'label', 'predicted')
em.print_eval_summary(lg_eval)

Precision : 100.0% (88/88)
Recall : 100.0% (88/88)
F1 : 100.0%
False positives : 0 (out of 88 positive predictions)
False negatives : 0 (out of 112 negative predictions)
